In [121]:
import pandas as pd
from collections import Counter
import statistics
import category_encoders as ce 

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time

In [22]:
df = pd.read_csv('https://firebasestorage.googleapis.com/v0/b/civic-abode-211408.appspot.com/o/hotels.csv?alt=media&token=aa9c8eac-6a57-4caa-bfa4-e5a3b198d2fd')

In [36]:
hotels = df.copy()
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [38]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [39]:
hotels['lat'] = hotels['lat'].fillna(statistics.median(hotels['lat']))
hotels['lng'] = hotels['lat'].fillna(statistics.median(hotels['lat']))


In [40]:
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.split(' ')[0])
hotels['days_since_review'] = hotels['days_since_review'].astype(int)
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_month'] = hotels['review_date'].dt.month

In [41]:
hotels['hotel_address'][11].split(' ')[-2:]
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: ' '.join(x.split(' ')[-2:]) if 'United Kingdom' in x else x.split(' ')[-1])
# hotels['hotel_city'] = hotels['hotel_address'].apply(lambda x: ''.join(x.split(' ')[-5]) if 'United Kingdom' in x else x.split(' ')[-2])

In [42]:
(hotels['reviewer_nationality'].value_counts(normalize=True).round(4)*100).head(10)

 United Kingdom               47.58
 United States of America      6.86
 Australia                     4.20
 Ireland                       2.88
 United Arab Emirates          1.97
 Saudi Arabia                  1.74
 Netherlands                   1.71
 Switzerland                   1.68
 Canada                        1.55
 Germany                       1.54
Name: reviewer_nationality, dtype: float64

In [43]:
bin_encoder = ce.BinaryEncoder(cols=['hotel_name', 'reviewer_nationality'])
hotels_bin = bin_encoder.fit_transform(hotels[['hotel_name', 'reviewer_nationality']])
hotels = pd.concat([hotels, hotels_bin], axis=1)

In [44]:
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,...,hotel_name_9,hotel_name_10,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7
0,Stratton Street Mayfair Westminster Borough Lo...,581,2016-02-19,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,...,0,1,0,0,0,0,0,0,0,1
1,130 134 Southampton Row Camden London WC1B 5AF...,299,2017-01-12,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,...,1,0,0,0,0,0,0,0,0,1
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,2016-10-18,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,...,1,1,0,0,0,0,0,0,1,0


In [45]:
hotels['hotel_country'].nunique()

encoder = ce.OneHotEncoder(cols=['hotel_country'])
country_bin = encoder.fit_transform(hotels['hotel_country'])
hotels = pd.concat([hotels, country_bin], axis=1)

In [46]:
hotels.drop(['hotel_address', 'hotel_name', 'reviewer_nationality', 'hotel_country'], axis=1, inplace=True)

In [48]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 40 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   additional_number_of_scoring                386803 non-null  int64         
 1   review_date                                 386803 non-null  datetime64[ns]
 2   average_score                               386803 non-null  float64       
 3   negative_review                             386803 non-null  object        
 4   review_total_negative_word_counts           386803 non-null  int64         
 5   total_number_of_reviews                     386803 non-null  int64         
 6   positive_review                             386803 non-null  object        
 7   review_total_positive_word_counts           386803 non-null  int64         
 8   total_number_of_reviews_reviewer_has_given  386803 non-null  int64        

In [72]:
hotels['tags'][0][3:-3].split(" ', ' ")

['Leisure trip',
 'Couple',
 'Studio Suite',
 'Stayed 2 nights',
 'Submitted from a mobile device']

In [83]:
tags_dict = {}
def get_tags_count(tag):
    tag = tag[3:-3].split(" ', ' ")
    for el in tag:
        if el in tags_dict:
            tags_dict[el] += 1
        else:
            tags_dict[el] = 1
    return

In [84]:
hotels['tags'].apply(get_tags_count)

0         None
1         None
2         None
3         None
4         None
          ... 
386798    None
386799    None
386800    None
386801    None
386802    None
Name: tags, Length: 386803, dtype: object

In [94]:
tags_df = pd.DataFrame(tags_dict, index=['count']).transpose()

In [113]:
top_tags = list(tags_df['count'].sort_values(ascending=False).index[:20])

In [116]:
for tag in top_tags:
    hotels[tag] = hotels['tags'].apply(lambda x: 1 if tag in x else 0)

In [118]:
hotels.drop(['tags'], axis=1, inplace=True)

In [120]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 59 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   additional_number_of_scoring                386803 non-null  int64         
 1   review_date                                 386803 non-null  datetime64[ns]
 2   average_score                               386803 non-null  float64       
 3   negative_review                             386803 non-null  object        
 4   review_total_negative_word_counts           386803 non-null  int64         
 5   total_number_of_reviews                     386803 non-null  int64         
 6   positive_review                             386803 non-null  object        
 7   review_total_positive_word_counts           386803 non-null  int64         
 8   total_number_of_reviews_reviewer_has_given  386803 non-null  int64        

In [123]:
nltk.downloader.download('vader_lexicon')

sent_analyzer = SentimentIntensityAnalyzer()

rev1 = "The hotel was very good, I love it!"
rev2 = "It was just horrible, the worst ever."

print(f"review 1:\n{rev1}\nScore: {sent_analyzer.polarity_scores(rev1)}")

print(f"\nreview 2:\n{rev2}\nScore: {sent_analyzer.polarity_scores(rev2)}")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maxmercury/nltk_data...


review 1:
The hotel was very good, I love it!
Score: {'neg': 0.0, 'neu': 0.386, 'pos': 0.614, 'compound': 0.8386}

review 2:
It was just horrible, the worst ever.
Score: {'neg': 0.603, 'neu': 0.397, 'pos': 0.0, 'compound': -0.8225}


In [124]:
hotels['negative_review']

0                                                  Leaving 
1                                            poor breakfast
2                                        No kettle in room 
3                                               No Negative
4                                              Torn sheets 
                                ...                        
386798                                          No Negative
386799                                          No Negative
386800     room is really small but guess is normal in L...
386801                                          No Negative
386802     Patio outside could have been cleaned of alga...
Name: negative_review, Length: 386803, dtype: object